<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

# CNN mit Keras: Hausnummern erkennen

In dieser Aufgabe wollen verwenden wir erneut den Datensatz, der Bildausschnitte mit einzelnen Ziffern von Hausnummern enthält.
Wir sind also wieder bei dem Problem der Klassifikation von Ziffern auf Grundlage von Bildern, wie schon beim klassischen MNIST Datensatz.
Da es sich aber hier um (Farb-) Fotos handelt, die zudem noch recht verrauscht sind, ist das Problem, die Ziffern zu erkennen, aber deutlich schwieriger.

Wir laden zunächst die Bidler von der URL http://ufldl.stanford.edu/housenumbers herunter.
Details zum Datensatz finden Sie in [1].

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
print("Lade Tensorflow in Version", tf.__version__)
from packaging import version
assert version.parse(tf.__version__) > version.parse("2.3.0"), \
    "TF 2.3.0 hat einen Bug beim Speichern und Laden von Modellen. Siehe [2]"
import os
import tarfile
import urllib.request



url = [f"http://ufldl.stanford.edu/housenumbers/{n}_32x32.mat" for n in ("train", "test")]
dfile = [f"./{n}_32x32.mat" for n in ("train", "test")]


for i in range(len(url)):
    if not os.path.isfile(dfile[i]):
        urllib.request.urlretrieve(url[i], dfile[i])

Die Daten liegen im `.mat`-Format vor, dass zumeist in Matlab verwendet wird.
Wir importieren die Daten über die Funktion `scipy.io.loadmat` und extrahieren dann die Attribute und Labels jeweils aus den Test- und Trainings-Daten.

In [ ]:
from scipy.io import loadmat
train_raw = loadmat('./train_32x32.mat')
test_raw = loadmat('./test_32x32.mat')
                   
train_images = np.array(train_raw['X'])
test_images = np.array(test_raw['X'])

train_labels = train_raw['y']
test_labels = test_raw['y']
                   
print(train_images.shape)
print(test_images.shape)

Wenn Sie sich die Dimension der Datensätze ansehen, stellen Sie fest, dass die Daten unpassend strukturiert sind.
In den ersten Dimensionen haben wir die (RGB) Pixel der einzelnen Bilder, in der letzten Dimension die einzelnen Bilder.
Daher sortieren wir die Dimensionen, bzw. die Axen unserer Datensätze um, sodass die erste Dimension dem Index eines Bildes entspricht.

In [ ]:
# Fix the axes of the images

train_images = np.moveaxis(train_images, -1, 0)
test_images = np.moveaxis(test_images, -1, 0)

print(train_images.shape)
print(test_images.shape)

Nun können wir ein zufälliges Bild ausgeben:

In [ ]:
import matplotlib.pyplot as plt
import random 
# Plot a random image and its label

plt.imshow(train_images[random.randint(0,len(train_images))])
plt.show()

print('Label: ', train_labels[13529])

In [ ]:
train_images = train_images.astype('float64')
test_images = test_images.astype('float64')

train_labels = train_labels.astype('int64')
test_labels = test_labels.astype('int64')

train_images /= 255.0
test_images /= 255.0

train_labels -= 1
test_labels -= 1

Wir stellen nun ein sequenziellen Keras Model auf. Das Modell hat 2 Schichten.
Die erste Schicht besitzt 128 Neuronen und verwendet ReLU als Aktivierungsfunktion.
Die zweite Schicht besitzt 10 Neuronen und verwendet als Aktivierungsfunktion die Softmax-Funktion.

In [ ]:
#Modell definieren

def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(32, 32, 3)))
    model.add(keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(keras.layers.Dense(10, activation=tf.nn.softmax))
    return model

model = create_model()

Nun wählen wir geeignete Parameter für das Modell aus

In [ ]:
#Modellparameter
optimizer =  'sgd'
loss = 'sparse_categorical_crossentropy'
metrics = ['accuracy']

#Modell erzeugen
model.compile(optimizer,loss,metrics)


Nun können wir das Modell trainieren.

In [ ]:

mname = "mlp_model"

if not os.path.exists(mname):
    print("Neues Modell anlegen!")
    model = create_model()
    model.compile(optimizer,loss,metrics)
else:
    print("Load Model from file", mname)
    model = tf.keras.models.load_model(mname)

history = model.fit(train_images, train_labels, epochs=5)
model.save(mname)



In [ ]:
#Trainiertes Modell auswerten
model = tf.keras.models.load_model("mlp_model")
test_loss, test_acc = model.evaluate (test_images, test_labels)
print('Classification Accuracy (Test):', test_acc)

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.title('Classification Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoche')
    plt.legend(['Test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.title('Kostenfunktion')
    plt.ylabel('Fehler')
    plt.xlabel('Epoche')
    plt.legend(['Test'], loc='upper left')
    plt.show()
    
plot_history(history)

**Aufgabe:** Erstellen Sie ein Faltungsnetz (CNN) mit folgenden Schichten über die *Keras Sequential API*:

1. Die erste Schicht soll ein `Conv2D`-Layer sein. Sie benötigen keinen zusätzlichen `Flatten`-Layer. Verwenden Sie in der ersten Schicht einfach den Parameter `input_shape` und setzen Sie ihn auf das 3-Tupel `(32, 32, 3)` (32x32 Pixel im RGB Format). Die `Conv2D`-Schicht soll 16 Filtermasken einer Größe von $3\times{}3$ Pixeln verwenden. Als Aktivierungsfunktion verwenden Sie `relu`.
2. Die zweite Schicht ist ein Pooling-Layer. Fassen Sie immer $2\times{}2$ Pixel mit Max-Pooling (`MaxPooling2D`) zusammen.
3. Um Overfitting zu vermeiden, führen wir ein Dropout-Layer ein. Verwenden die eine Dropout-Rate von 30%.
4. Fügen Sie je eine weitere `Conv2D`, `MaxPooling2D` und `Dropout` Schicht ein. Die Parameter sollen wie bei 1.-3. gewählt werden, mit der Ausnahme, dass die `Conv2D`-Schicht nun doppelt so viele (also 32) Filtermasken verwenden soll.
5. Transformieren Sie die 2D-Daten in 1D- Daten mit einem `Flatten`-Layer. `Flatten` benötigt keine Parameter.
6. Fügen Sie dem Modell eine voll-vermaschte Schicht (`Dense`) mit 256 Neuronen hinzu. Als Aktivierungsfunktion soll hier acuh `relu` verwendet werden.
7. Fügen Sie ein Dropout Schicht mit 30% Dropout rate ein
8. Fügen Sie dem Modell einen Eine Ausgabeschicht mit 10 Neuronen hinzu (`Dense`). Die Aktivierungsfunktion hier sollte `softmax` sein.

In [ ]:
#Modell definieren
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

def create_model():
    model = keras.Sequential()
    # YOUR CODE HERE
    raise NotImplementedError()
    return model

model = create_model()

Nun trainieren wir das Modell und schreiben die Trainierten Parameter erneut in eine Datei.

(*Hinweis:* Achten Sie darauf, die Datei zu löschen, falls Sie ein neues Modell erzeugt haben)

In [ ]:
mname = "cnn_model"

if not os.path.exists(mname):
    print("Neues Modell anlegen!")
    model = create_model()
    model.compile(optimizer,loss,metrics)
else:
    print("Load Model from file", mname)
    model = tf.keras.models.load_model(mname)

history = model.fit(train_images, train_labels, epochs=5)
model.save(mname)

Wir können nun das trainierte Modell bewerten.

In [ ]:
#Trainiertes Modell auswerten
model = tf.keras.models.load_model("cnn_model")
test_loss, test_acc = model.evaluate (test_images, test_labels)
print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.title('Classification Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoche')
    plt.legend(['Test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.title('Kostenfunktion')
    plt.ylabel('Fehler')
    plt.xlabel('Epoche')
    plt.legend(['Test'], loc='upper left')
    plt.show()
    
plot_history(history)

### Referenzen

[1] Yuval Netzer, Tao Wang, Adam Coates, Alessandro Bissacco, Bo Wu, Andrew Y. Ng. *"Reading Digits in Natural Images with Unsupervised Feature Learning"*,  NIPS Workshop on Deep Learning and Unsupervised Feature Learning 2011.

[2] Tensorflow Github Issue #42459 [Accuracy is lost after save/load #42459](https://github.com/tensorflow/tensorflow/issues/42459)
